Импорт библиотек

In [1]:
from scipy import stats as st
import pandas as pd
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Импорт данных из файла:

In [2]:
games = pd.read_csv('/games.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/games.csv'

**1. Изучение данных**

In [ ]:
games.sample(5)

имеет смысл привести названия столбцов к нижнем регигстру;
рейтинги и оценки выражаются не только числами, но и строковыми данными;
очевидно, будут пропуски - как минимум, в оценках/рейтингах.

**2. Подготовка данных**

Приведем названия столбцов к нижнему регистру

In [ ]:
games.columns = games.columns.str.lower()

Изучим типы данных

In [ ]:
games.info()

Рассмотрим подробнее "name", "platform", "genre", "rating" к типу "строка".

In [ ]:
type(games['name'][0])

In [ ]:
type(games['platform'][0])

In [ ]:
type(games['genre'][0])

In [ ]:
type(games['rating'][0])

Четыре колонки с типом "object" оказались cтроками. С ними ничего не делаем. Перед приведением типов данных в "year of release" и "user score" изучим состав данных и пропуски.

В колонках "name" и "genre" есть два пропуска (16713 значений при размере таблицы 16715). Рассмотрим соответствующие строки.

In [ ]:
games.loc[games['name'].isna()]

В этих строках отсутствуют названия игр и данные по оценкам/рейтингу. Тем не менее, есть данные о платформе и продажах, 
которые учтутся при анализе продаж по платформам. Строки не удаляем. Заменим название и жанр на "unknown".

In [ ]:
games.loc[games['name'].isna(), 'name'] = 'unknown'
games.loc[games['genre'].isna(), 'genre'] = 'unknown'

Рассмотрим строки с пропущенными годами выспуска.

In [ ]:
games.loc[games['name'].isna()]

In [ ]:
len(games.loc[games['year_of_release'].isna()])/len(games)

Пропусков менее 2%. Имеем ввиду, что ввиду малого количества можно удалить соответствующие строки. Но пока не удаляем,
так как в данных строках есть информация о продажах и рейтингах. Данные строки не примут участие в анализе распределения продаж по годам, но могут повлиять на резултьтат при анализе распределения по платформам.

Рассмотрим столбец "user score". Аббревиатура "tbd" может означать "to be determined" или "to be deleted". В любом случае, согласно заданию проекта дальнейшая работа с рейтингами будет вестись как с числами. Поэтому, cтроки с "tbd" не будут принимать участие в соответствующем анализе.
Тем не менее, NAN и tbd в строках с оценками/рейтингами пока не удаляем.
Поскольку имеются строки с пропусками одной оценки, но присутствующей другой оценкой, а также ввиду небольшого размера датафрейма, более объектиным подходом будет удалять их каждый раз перед операциями, в которых оценки/рейтинги влияют на результат. И после удаления преобразовывать оставшиеся данные в колонке в числовой формат.

Рассчитаем суммарные продажи во всех регионах и запишем их в отдельный столбец.

In [ ]:
games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']

**3. Исследовательских анализ данных**

**3.1 Количество выпущенных игр в разные годы**

In [ ]:
def games_years_creation(df):
    games_years = df
    games_years['year_of_release'].fillna(0)
    games_years = games_years.loc[games_years['year_of_release'] >0]
    return games_years
games_years = games_years_creation(games)
games_years = games_years.groupby('year_of_release')['year_of_release'].count().to_frame()
games_years['year'] = games_years.index
games_years = games_years.rename(columns = {'year_of_release': 'quantiy'})
fig= px.bar(games_years, y = 'quantiy',title="Количсество выпущенных игр по годам")
fig.update_layout(xaxis_title='Год выпуска',yaxis_title='Количество выпущенных игр')
fig.show()

В задании Проекта есть вопрос "за все ли периоды важны данные?". В аспекте построения прогноза на 2017 год точно не важны данные ранее 1995го. Если бы рост был постоянным, то "начало" до 1995 можно было бы учесть. С помощью них уточнились бы выводы. Но поскольку имеются периоды спада, особенно резкое падение после 2010 года, и с учетом разницы на порядок между данными 90х и 2010х, то

1) наиболее вероятно, что на взлеты/падения в 2010е влияют уже не те факторы, что в 80-90е и нет смысла учитывать соответствующие данные; 2) даже если факторы одни и те же, отбрасывание данных до 1995г не внесет существенных изменений в результат.

В заданиии требуется выбрать актуальный период. Возьмем 1995-2016 гг. Взлет и падение попробуем учесть. 

**3.2 Продажи на самых популярных платформах по годам**

Выделим платформы с наибольшими суммарными продажами.

In [ ]:
def platformer(df):
    games_platforms=df.groupby('platform')['total_sales'].sum().to_frame().sort_values(by='total_sales', ascending=False).reset_index()
    return games_platforms
games_platforms = platformer(games)
games_platforms.head(20)

Обнаружена разница более чем в два раза между PS (730) и GBA (318). Границу между "самыми популярными" и "остальными"
платформами проведем именно в этом месте. Среди "самых популярных" и первых десяти "остальных" разница между между продажами меняется не сильно по сравнению со скачком от 318 до 730. Следовательно верхнюю группу (730 и выше) стоит рассматривать как объект схожей природы, а "остальные" - отдельно. 

Создадим новый датафрейм с названиями ТОП-6 площадок с наибольшими продажами.

In [ ]:
games_platforms_top = games_platforms.iloc[0:6,0:1]
games_platforms_top['platform_number'] = games_platforms_top.index
games_platforms_top

К таблице с наименованиями наиболее популярных платформ присоединим таблицу с продажами по годам. Построим графики.

К таблице с наименованиями наиболее популярных платформ присоединим таблицу с продажами по годам. Построим графики.

In [ ]:
games_years = games_years_creation(games)
games_years = games_years.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()
games_years_platforms = games_platforms_top.set_index(['platform']).join(games_years.set_index(['platform']))
fig = make_subplots(rows=6,cols=1)
names = games_platforms_top.iloc[:,0].tolist()
for i in range(len(games_platforms_top)):
    f = games_years_platforms.loc[games_years_platforms['platform_number'] == i]
    x = f['year_of_release'].tolist()
    y = f['total_sales'].tolist()
    fig.add_bar(x=x, y=y, row=i+1, col = 1, name=names[i])
    fig.update_xaxes(title_text="Год", row=i+1, col=1)
    fig.update_yaxes(title_text="Суммарные продажи", row=i+1, col=1)
fig.update_layout(height=2000, width=1000, title_text = 'Годовые продажи на популярных платформах')
fig.show() 

В задании есть требование выделить характерный срок, за который появляются и исчезают платформы. По ТОП-6 можно сказать, что этот срок составляет 6-9 лет. 

**3.3 Выбор потенциально прибыльных платформ**

Выделим "растущие" платформы.

In [ ]:
len(games_platforms)

Всего платформ 31. Попробуем отсмотреть графики годовых продаж по каждой платформе, и выделить растущие. Расположим все на одном графике. Неактуальный период до 1995 года исключим из рассмотрения.

In [ ]:
games_platforms = games_platforms.iloc[:,0:1]
def plot_year_platforms(margin_year):
    games_platforms['platform_number'] = games_platforms.index
    games_years = games_years_creation(games)
    games_years = games_years.loc[games_years['year_of_release'] >= margin_year]
    games_years = games_years.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()
    games_years_platforms = games_platforms.set_index(['platform']).join(games_years.set_index(['platform']))
    games_years_platforms['platform']=games_years_platforms.index
    fig = px.line(games_years_platforms, x="year_of_release", y="total_sales", color = 'platform')
    fig.update_layout(xaxis_title='Год выпуска',yaxis_title='Суммарные продажи')
    fig.show()
plot_year_platforms(1995) 

Растущих платформ на таком графике не видно. Попробуем сузить интервал рассмотрения в годах.

In [ ]:
plot_year_platforms(2010)

Растущих платформ опять не видно. Топ 5 платформ падают. Попробуем рассмотреть без Топ-5 интервал 2010 - 2016 гг.

In [ ]:
games_platforms = platformer(games)
games_platforms = games_platforms.iloc[5:30,0:1]
plot_year_platforms(2010)

Среди нетоповых платформ также все падают. Очевидно, есть сильные внешние факторы, влияющие абсолютно на все платформы, и нет ни одной платформы, которая их поборола бы. Поэтому в качестве потенциально прибыльных принимаем ТОП-5 по сумме 2014 - 2016го года. Взятие интервала 2014-2016 вместо одного года 2016го обусловлено тем, что интервал 2014-2016 учтет помимо самых свежих данных еще и недавний пик 2015 года, имевший место для значительного количества платформ. Пик 2015го следует учесть поскольку 1) не все компании имеют этот пик 2) размер пика показывает, как каждая из платформ смогла использовать благоприятные условия 2015 года. 
Также, в задании проекта сообщается, что данные за 2016 год могут быть неполными. Поэтому особенно важно рассмотреть также и 2014-2015 годы.

In [ ]:
actual_platforms = games.loc[games['year_of_release']>=2014]
actual_platforms = actual_platforms.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()
actual_platforms = actual_platforms.groupby(['platform'])['total_sales'].sum().reset_index().sort_values(by='total_sales',ascending = False)
actual_platforms = actual_platforms.loc[0:5,:].reset_index()
actual_platforms = actual_platforms.loc[:,'platform'].tolist()
actual_platforms

In [ ]:
from matplotlib.pyplot import figure
platforms_for_boxplot = games.loc[games['year_of_release']>=2014]
platforms_for_boxplot = platforms_for_boxplot.loc[:,['platform','total_sales']]
p = []
for i in range(len(actual_platforms)):
    p.append(platforms_for_boxplot.loc[platforms_for_boxplot['platform'] == actual_platforms[i]])     
fig, axs= plt.subplots(3,2,figsize=(6,12))
axs[0,0].boxplot(p[0].iloc[:,1])
axs[0,0].set_title(actual_platforms[0])
axs[0,1].boxplot(p[1].iloc[:,1])
axs[0, 1].set_title(actual_platforms[1])
axs[1, 0].boxplot(p[2].iloc[:,1])
axs[1, 0].set_title(actual_platforms[2])
axs[1, 1].boxplot(p[3].iloc[:,1])
axs[1, 1].set_title(actual_platforms[3])
axs[2, 0].boxplot(p[4].iloc[:,1])
axs[2, 0].set_title(actual_platforms[4])
axs[2, 1].boxplot(p[5].iloc[:,1])
axs[2, 1].set_title(actual_platforms[5])
fig.subplots_adjust(left=0.08, right=2, bottom=0.2, top=3,
                    hspace=0.2, wspace=0.6)
plt.show()

In [ ]:
games['year_of_release'].unique()

In [ ]:
actual_platforms_df = pd.DataFrame(actual_platforms)
actual_platforms_df['platform_number'] = actual_platforms_df.index
actual_platforms_df = actual_platforms_df.set_index(0)
platforms_for_boxplot = platforms_for_boxplot.set_index('platform').join(actual_platforms_df)
platforms_for_boxplot = platforms_for_boxplot.loc[platforms_for_boxplot['platform_number'] <= 5]
platforms_for_boxplot = platforms_for_boxplot.reset_index()

In [ ]:
platforms_for_boxplot

In [ ]:
import seaborn as sns

plt.figure(figsize=(20, 10))

sns.boxplot(data=platforms_for_boxplot, x='index', y='total_sales')

plt.ylim(0, 2.0)

plt.title('"Ящики с усами" по глобальным продажам игр')
plt.xlabel('Продажи, млн. шт.')
plt.ylabel('Платформы')

plt.show()

бОльшая часть значений лежит в диапазоне до 350. Медиана на уровне 200. ТОП-6, для которых уже построены столбчатые диаграмы выше, являются выбросами.

Для всех шести выбранных актуальными платформ данные скошены в сторону бОльших продаж. "Ящики" несимметричны относительно медиан. И во всех случаях имеются выбросы сверх полутора межквартильных расстояний.

**3.5 Влияние отзывов на продажи внутри одной платформы**

Рассмотрим платформу 3DS, лидирующую по суммарным продажам за период 2014-2016.

In [ ]:
platform_3DS = games.loc[games['platform'] == '3DS']
platform_3DS = pd.DataFrame(platform_3DS)
platform_3DS = platform_3DS.loc[platform_3DS['year_of_release'] >= 2014]
platform_3DS.info()

Построим для выбранной платформы диаграму рассеяния "оценки критиков - суммарные продажи по всем регионам". NAN в оценках пользователей исключим из рассмотрения.

In [ ]:
def one_platform_NAN_deleting(df):
    platform_3DS = df
    platform_3DS['critic_score'].fillna(0)
    platform_3DS = platform_3DS.loc[platform_3DS['critic_score'] >0]
    return platform_3DS
platform_3DS = one_platform_NAN_deleting(platform_3DS)

In [ ]:
plt.figure(figsize=(30, 3))
fig, ax = plt.subplots(figsize=(12, 8))
plt.xlabel("Оценка критиков") 
plt.ylabel("Продажи по всем регионам")
plt.title("Диаграмма рассеяния: оценки критиков - суммарные продажи по всем регионам")
ax.scatter(x=platform_3DS['critic_score'],y= platform_3DS['total_sales'])

График демонстрирует связь оценки критиков и продаж. Чем выше оценка, тем больше продажи. Тем не менее, это соответствие обеспечивается, в основном, выбросами. Рассчитаем коэффициент корреляции Пирсона между отзывами критиков и продажами.

In [ ]:
print(platform_3DS['critic_score'].corr(platform_3DS['total_sales']))

Характер связи между рассматриваемыми величинами, очевидно, линейный. Поэтому, коэффициент Пирсона можно считать адекватной количественной характеристикой данной связи. Коэффициент Пирсона положительный - это означает, что с ростом оценки критиков растут и продажи. Тем не менее, связь не очень "тесная". 
Оба вывода соответствуют наблюдаемой на графике картине.

Построим для выбранной платформы диаграму рассеяния "оценки пользователей - суммарные продажи по всем регионам". NAN в оценках пользователей исключим из рассмотрения.

In [ ]:
platform_3DS = games.loc[games['platform'] == '3DS']
platform_3DS = platform_3DS.loc[platform_3DS['year_of_release'] >= 2014]
platform_3DS = platform_3DS.loc[:,['platform','user_score','total_sales']].dropna()
platform_3DS = platform_3DS.loc[platform_3DS['user_score'] != 'tbd'].sort_values(by = 'user_score')
platform_3DS[['total_sales','user_score']]  = platform_3DS[['total_sales','user_score']].astype(float)

In [ ]:
plt.figure(figsize=(30, 3))
fig, ax = plt.subplots(figsize=(20, 8))
plt.xlabel("Оценка пользователей") 
plt.ylabel("Продажи по всем регионам")
plt.title("Диаграмма рассеяния: оценки пользователей - суммарные продажи по всем регионам")
import matplotlib.ticker as ticker
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax.scatter(x=platform_3DS['user_score'],y= platform_3DS['total_sales'],vmin=2, vmax=10)

График демонстрирует связь оценки пользователей и продаж. Чем выше оценка, тем больше продажи. Как и в случае с оценкой критиков соответствие обеспечивается, в основном, выбросами. Рассчитаем коэффициент корреляции Пирсона между оценками пользователей и продажами.

In [ ]:
print(platform_3DS['user_score'].corr(platform_3DS['total_sales']))

Положительная корреляция между оценками пользователя и суммарными продажами подтверждается и графиком и коэффициентом Пирсона. Но в случае оценки пользователей эта связь еще "слабее".

**3.6 Сопоставление платформ в аспекте связи между оценками и продажами**

Рассчитаем коэффициенты Пирсона для всех платформ.

In [ ]:
def Pirson_for_critic_score(platform):
    platform = games.loc[games['platform'] == platform]
    platform_3DS = platform.loc[platform['year_of_release'] >= 2014]
    platform['critic_score'].fillna(0)
    platform = platform.loc[platform['critic_score'] >0]
    P = platform['critic_score'].corr(platform['total_sales'])
    return P
def Pirson_for_user_score(platform):
    platform = games.loc[games['platform'] == platform]
    platform_3DS = platform.loc[platform['year_of_release'] >= 2014]
    platform = platform.loc[:,['platform','user_score','total_sales']].dropna()
    platform = platform.loc[platform['user_score'] != 'tbd'].sort_values(by = 'user_score')
    platform[['total_sales','user_score']]  = platform[['total_sales','user_score']].astype(float)
    P = platform['user_score'].corr(platform['total_sales'])
    return P
Pirson_for_user_score('X360')

In [ ]:
len(actual_platforms)

In [ ]:
P1_list = []
P2_list = []
for i in range(len(actual_platforms)):
    P1_list.append(Pirson_for_critic_score(actual_platforms[i]))
    P2_list.append(Pirson_for_user_score(actual_platforms[i]))    
Pirson_table = pd.DataFrame({'platform' : actual_platforms, 'pirson_critic' : P1_list, 'pirson_user': P2_list})
display(Pirson_table)

Наблюдается слабая положительная корреляция по всем платформам кроме одной (PC). Плафтормы отсортированы по выручке. Наблюдается тенденции "больше выручка - сильнее корреляция продаж с отзывами". Тем не менее, этот тренд нарушает находящаяся на четвертом месте платформа WiiU.

Проверка показывает, что вввиду малого количества продаж на данных платформах, оценки критиков и пользователей по продаваемым на них играх полностью отсутствуют.

**3.6 Распределение игр по жанрам**

Создадим таблицу со столбцами "жанр","суммарные продажи","количество игр","продажи/количество"

In [ ]:
genre_quantity_count = games.loc[games['year_of_release'] >= 2014]
actual_platforms_df = pd.DataFrame(actual_platforms)
actual_platforms_df['platform_number'] = actual_platforms_df.index
actual_platforms_df = actual_platforms_df.set_index(0)
genre_quantity_count = genre_quantity_count.set_index('platform').join(actual_platforms_df)
genre_quantity_count = genre_quantity_count.loc[genre_quantity_count['platform_number'] <= 5]
genre_quantity_count = genre_quantity_count.groupby('genre')['name'].count().to_frame()
genre_quantity_count = genre_quantity_count.rename(columns = {'name' : 'quantity'})
genre_sales_count = games.groupby('genre')['total_sales'].sum().to_frame()
genre_analysis = genre_quantity_count.join(genre_sales_count)
genre_analysis['ratio'] = round(genre_analysis['total_sales']/genre_analysis['quantity'],2)
genre_analysis['genre'] = genre_analysis.index
genre_analysis = genre_analysis.sort_values(by = 'ratio', ascending = False)
genre_analysis

Наилучшее соотношение суммарных продаж к количеству выпущенных игр - у жанра "Platform". Жанр "Action", лидирующий по 
количеству игр находится лишь в середине рейтинга по соотношению выручки и количеству игр.
Построим круговую диаграму распределения по жанрам.

In [ ]:
fig = px.pie(genre_analysis, values='quantity', names='genre', title='Распределение выпущенных игр по жанрам')
fig.show()

В задании присутствует вопрос "выделяются ли жанры с высокими и низкими продажами?". Ответить можно так: 1) Разница между лидирующими по соотношению "выручка/количество" играми и аутсайдерами  - примерно в 15 раз (на порядок). 2) При этом не выделяется двух или более групп с "высокими" или "низкими" продажами. Коэффициент продажи/количество распределен по жанрам равномерно. 

Что касается распределения игр по жанрам - примерно половину количества игр занимают ТОП-2 жанров. Оставшиеся распределены с равномерным убыванием доли.

**4. Портрет пользователя каждого региона**

**4.1 ТОП-5 платформ каждого региона по продажам**

In [ ]:
games_for_region_analisys = games.loc[games['year_of_release'] >= 2014]
games_for_region_analisys = games_for_region_analisys.set_index('platform').join(actual_platforms_df)
games_for_region_analisys = games_for_region_analisys.loc[games_for_region_analisys['platform_number'] <= 5]
games_for_region_analisys = games_for_region_analisys.reset_index().rename(columns = {'index':'platform'})

In [ ]:
def top5_region(df, region):
    top5 = df.loc[:,['platform', region]].groupby('platform').sum().sort_values(by=region, ascending = False).iloc[0:5,:]
    return top5
na_top5 = top5_region(games_for_region_analisys, 'na_sales').reset_index()
eu_top5 = top5_region(games_for_region_analisys, 'eu_sales').reset_index()
jp_top5 = top5_region(games_for_region_analisys, 'jp_sales').reset_index()
other_top5 = top5_region(games_for_region_analisys, 'other_sales').reset_index()
display(na_top5,eu_top5,jp_top5,other_top5)

В двух регионах лидирует платформа PS3, она же присутствует во всех ТОП-5. В Японии платформа 3DS лидирует с отрывом в 3 раза от ближайшего конкурента. Более ни в одном из регионов столь значительных разностей между соседними строками рейтингов не встречается, за исключением нижних строчек Японского рейтинга и рейтинга по странам "остального мира".

**4.2 ТОП-5 жанров каждого региона**

In [ ]:
def top5_genre_region(df, region):
    top5 = df.loc[:,['genre', region]].groupby('genre').sum().sort_values(by=region, ascending = False).iloc[0:5,:]
    return top5
na_top5 = top5_genre_region(games_for_region_analisys, 'na_sales').reset_index()
eu_top5 = top5_genre_region(games_for_region_analisys, 'eu_sales').reset_index()
jp_top5 = top5_genre_region(games_for_region_analisys, 'jp_sales').reset_index()
other_top5 = top5_genre_region(games, 'other_sales').reset_index()
display(na_top5,eu_top5,jp_top5,other_top5)

Жанр Action лидирует во всех регионах кроме Японии, где он на втором месте. Следует отметить в Японском рейтинге две выделаюшиеся группы: Первая пара "Role Playing + Action" с огромным отрывом в 4 раза по продажам опережает "отстающую" тройку. Больше столь радикальных разрывов во всех рейтингах не наблюдается.

**4.3 Влияние рейтинга ESRB на продажи в каждом регионе**

Отсутствующие значения рейтинга "NAN" заполняем условным символом "No_Rating" и оставляем.

In [ ]:
games_for_rating_analysis = games.loc[:,['year_of_release', 'rating','na_sales','eu_sales','jp_sales','other_sales','platform']]
games_for_rating_analysis = games_for_rating_analysis.set_index('platform').join(actual_platforms_df)
games_for_rating_analysis = games_for_rating_analysis.loc[games_for_rating_analysis['platform_number'] <= 5]
games_for_rating_analysis = games_for_rating_analysis.reset_index().rename(columns = {'index':'platform'})
games_for_rating_analysis['rating'].fillna('No_Rating',inplace=True)
games_for_rating_analysis

In [ ]:
rating_ammount = games_for_rating_analysis.groupby(['rating', 'year_of_release'])['na_sales'].count().to_frame().rename(columns = {'na_sales': 'amount'})
rating_ammount

Мы видим 8 различных значений рейтинга и одну группу игр "без рейитнга". 
Выделяются две группы оценок: часто встречающиеся оценки и редко встречающиеся. Имеется разница на два порядка между этими двумя группами по количеству получивших оценку игр. 
Подготовим новый датафрейм для построения графиков рейтинг-продажи для каждого региона.

In [ ]:
games_for_rating_sales_analysis = games_for_rating_analysis.groupby('rating').sum().reset_index()

In [ ]:
fig = make_subplots(rows=4,cols=1)
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['na_sales'], row=1, col = 1, name='Продажи в Северной Америке')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['eu_sales'], row=2, col = 1, name='Продажи в Европе')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['jp_sales'], row=3, col = 1, name='Продажи в Японии')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['jp_sales'], row=4, col = 1, name='Продажи в других регионах')
fig.update_layout(height=1000, width=1000, title_text = 'Продажи игр с различными рейтингами')
fig.show()

Количество игр "без рейтинга" - больше половины. Но по суммарным продажам в Северной Америке и Европе они существенно не выделяются. 
Однако в Японии и в "других" частях мира игры без рейтинга имеют продажи примерно в 4 раза больше, чем самые продаваемые группы игр "с рейтингом". Предположительно, это связано с наличием локальных рейтингов - более актуальных для местных покупателей чем мировые рейтинги.  
В Европе и Северной Америке лидируют игры с рейтингом М (от 17 лет) - для взрослых.

In [ ]:
games_for_rating_analysis = games.loc[:,['year_of_release','rating','na_sales','eu_sales','jp_sales','other_sales','platform']]
games_for_rating_analysis = games_for_rating_analysis.loc[games_for_rating_analysis['year_of_release'] >= 2014]
games_for_rating_analysis = games_for_rating_analysis.set_index('platform').join(actual_platforms_df)
games_for_rating_analysis = games_for_rating_analysis.loc[games_for_rating_analysis['platform_number'] <= 5]
games_for_rating_analysis = games_for_rating_analysis.reset_index().rename(columns = {'index':'platform'})
games_for_rating_analysis['rating'].fillna('No_Rating',inplace=True)
games_for_rating_analysis

In [ ]:
games_for_rating_sales_analysis = games_for_rating_analysis.groupby('rating').sum().reset_index()

In [ ]:
fig = make_subplots(rows=4,cols=1)
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['na_sales'], row=1, col = 1, name='Продажи в Северной Америке')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['eu_sales'], row=2, col = 1, name='Продажи в Европе')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['jp_sales'], row=3, col = 1, name='Продажи в Японии')
fig.add_bar(x=games_for_rating_sales_analysis['rating'], y=games_for_rating_sales_analysis['jp_sales'], row=4, col = 1, name='Продажи в других регионах')
fig.update_layout(height=1000, width=1000, title_text = 'Продажи игр с различными рейтингами')
fig.show()

В Японии и в "других" частях мира игры без рейтинга имеют продажи примерно в 4 раза больше, чем самые продаваемые группы игр "с рейтингом". Предположительно, это связано с наличием локальных рейтингов - более актуальных для местных покупателей чем мировые рейтинги.  
В Европе и Северной Америке лидируют игры с рейтингом E - для всех. В целом, в Европе и Северной Америке разброс по количеству игр с различными рейтингами не очень большой - разница между минимумом и максимумом примерно в два раза. 
Стоит отметить, что ни в одном регионе в актуальный датафрейм не попали игры с рейтингами "EC","KA" и "RP". 

**5. Проверка гипотез**

**5.1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые**

Нулевая гипотеза: "Средний пользовательский рейтинг платформ Xbox One равен среднему пользовательскому рейтингу
платформы PC". 

In [ ]:
def platform_rating_analysis(platform):
    platform = games.loc[games['platform'] == platform]
    platform = platform.loc[platform['year_of_release'] >= 2014]
    platform = platform.loc[:,['user_score']].dropna()
    platform = platform.loc[platform['user_score'] != 'tbd'].astype(float)
    return platform 
XOne_ratings = platform_rating_analysis('XOne')
PC_ratings = platform_rating_analysis('PC')

Проведем t-тест о равенстве средних двух генеральных совокупностей.

In [ ]:
results = st.ttest_ind(XOne_ratings, PC_ratings)
print('p-значение:', results.pvalue)

Даже если принять критический уровень статистической значимости Альфа равный 5%, он не будет превышен. Нулевая гипотеза не отвергается.

**5.2 Средние пользовательские рейтинги жанров Action и Sports разные**

Нулевая гипотеза: "Cредний пользовательский рейтинг игр жанров Action и Sports равны".

In [ ]:
def genre_rating_analysis(genre):
    platform = games.loc[games['genre'] == genre]
    platform = platform.loc[platform['year_of_release'] >= 2014]
    platform = platform.set_index('platform').join(actual_platforms_df)
    platform = platform.loc[platform['platform_number'] <= 5]
    platform = platform.loc[:,['user_score']].dropna()
    platform = platform.loc[platform['user_score'] != 'tbd'].astype(float)
    return platform 
Action_ratings = genre_rating_analysis('Action')
Sports_ratings = genre_rating_analysis('Sports')
results = st.ttest_ind(Action_ratings, Sports_ratings)
print('p-значение:', results.pvalue)

Даже если принять критичекий уровень статистической значимости (альфа) равным 1%, он не будет превышен. Нулевая гипотеза отвергается.

**6. Общий вывод**
 - Продажи игр в мире падают. Даже если данные за 2016 год неполные, тем не менее, всплеск 2015 года есть лишь у некоторых платформ;
 - Отзывы и коррелируют с продажами. Выше рейтинг - больше продаж. Корреляция слабая;
 - Для актуального в плане прогноза продаж на 2017 год датафрейма (6 платформ, период 2014-2016) все данные по суммарным продажам "скошены" в сторону бОльших продаж, и присутствует значительное количество выбросов;
 - Япония существенно отличается от всего мира. Выраженное предпочтенние ролевых игр, вероятно, является следствием их "загадочного" менталитета;
 - Также, в Японии и "других частях мира", предположительно, наиболее актуальными для покупателей являются местные рейтинги игр, которым они доверяют больше чем ESRB. Данное предположение сделано на основании того, что игры "без рейтинга ESRB" лидируют с отрывом в разы по отношению к играм "с рейтингом ESRB" в Японии и "других частях мира";
 - В актуальный период на актуальных платформах в Европе и Северной Америке лидируют игры категории "для всех". Этот факт следует учесть при планировании бизнеса в 2017 году;
 - Игры с рейтингами ESRB "EC",'KA" и "RP" вообще не продавались на актуальных платформах в 2014-2016 годах;
 - В актуальный период на актуальных платформах преобладают игры "Role Playing" и "Action". На их долю приходится половина всех продаж;
 - Cреди платформ заметно выделяются две группы: "ТОП-6" (выбросы) и "остальные". Результат характерен для крупного мирового бизнеса, когда несколько гигантов "делают" подавляющую часть выручки.